In [57]:
# How to get layer names passed in as URL parameters.
# 
# A button in the catalog can be programmatically built in this way
# to link to this jupyter notebook wherever it is served and have
# the notebook constructed so that we pre-select the target layer.
#
# https://stackoverflow.com/a/37134476

from IPython.display import HTML
HTML('''
    <script type="text/javascript">
        IPython.notebook.kernel.execute("URL = '" + window.location + "'")
    </script>''')


In [60]:
#print(URL)

In [16]:
import json
import threading
import logging
logging.basicConfig(level=logging.INFO)

LOGGER = logging.getLogger(__name__)

import pygeoprocessing
import shapely.geometry
from osgeo import gdal
import ipyleaflet
from ipyleaflet import Map, LocalTileLayer, basemap_to_tiles, basemaps
from ipyleaflet import LayersControl, ZoomControl, ScaleControl, LegendControl, WidgetControl, DrawControl
import IPython.display
import ipywidgets as widgets

gdal.DontUseExceptions()


# From https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html
class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to an output widget """

    def __init__(self, *args, **kwargs):
        super(OutputWidgetHandler, self).__init__(*args, **kwargs)
        layout = {
            'width': '100%', 
            'height': '160px', 
            'border': '1px solid black'
        }
        self.out = widgets.Output(layout=layout)

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        new_output = {
            'name': 'stdout', 
            'output_type': 'stream', 
            'text': formatted_record+'\n'
        }
        self.out.outputs = (new_output, ) + self.out.outputs
        
    def show_logs(self):
        """ Show the logs """
        display(self.out)
    
    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()
        
output_handler = OutputWidgetHandler()
logging.getLogger().addHandler(output_handler)


with open('stats.json') as stats_json:
    stats = json.load(stats_json)
    
minimum = stats['minimum']
maximum = stats['maximum']


chosen_basemap = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
chosen_basemap.name = '(basemap) OpenStreetMap Mapnik'

m = Map(
    basemap=chosen_basemap,
    center=(0,0),
    zoom=2,  # zoom level for most of the globedisplay(m)
)  

scalebar = ScaleControl(position='bottomleft')
m.add_control(scalebar)

# If we want to pre-cache these tiles on GCS, use ipyleaflet.leaflet.TileLayer instead
country_id_tiles_layer = LocalTileLayer(
    path='tiles/{z}/{x}/{y}.png',
    name='NASA SRTM',
    show_loading=True,
    attribution='NASA'
    
)
m.add_layer(country_id_tiles_layer)

layerlist = LayersControl(position='topleft')
m.add_control(layerlist)

# build a colormap from our GDAL-compatible GRASS color file
import branca
colors = []
with open('elevation-color-profile-grass.txt') as color_profile:
    for line in color_profile:
        try:
            percent, r, g, b = [c.strip() for c in line.split(' ') if c]
        except ValueError:
            pass
        colors.append((float(r), float(g), float(b)))

colormap = branca.colormap.LinearColormap(
    colors=colors,
    vmin=minimum, vmax=maximum)
colormap.caption = 'Elevation (m)'
html = f"{colormap._repr_html_()}"

colorbar = WidgetControl(widget=widgets.HTML(html), position='topright')
m.add_control(colorbar)


draw_control = DrawControl(polyline={}, circlemarker={})
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 0.5
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oups!"
    },
    "allowIntersection": False
}
m.add_control(draw_control)

submit_button = widgets.Button(
    description = 'Clip layer to AOI',
    disabled=False,  # start out disabled, enable when geometries defined
    button_style='',  # change to 'success' when geometries added
    tooltip='Clip the underlying layer by the bounding box of the geometries defined.',
    icon='scissors',  # this is a fontawesome icon
)
submit_button_control = WidgetControl(widget=submit_button, position='bottomright')


def _add_submit_button(control, **kwargs):
    # enable the submit button if 
    if kwargs['geo_json']['geometry']['coordinates']:
        m.add_control(submit_button_control)
    else:
        try:
            m.remove_control(submit_button_control)
        except ipyleaflet.ControlException:
            # When the control is not currently on the map
            pass
        
draw_control.on_draw(_add_submit_button)


def compute(source_raster_path, aoi_geom, target_epsg, target_raster_path):
    # TODO: if the target coordinate system is projected, use the user's pixel size
    #       It seems potentially challenging if we were to derive an appropriate linear pixel size,
    #       so let's just ask.
    LOGGER.info('bar')
    LOGGER.info('Warping')
    # Clip the source raster to the target bounding box in WGS84
    raster_info = pygeoprocessing.get_raster_info(source_raster_path)
    pygeoprocessing.warp_raster(
        base_raster_path=source_raster_path,
        target_pixel_size=raster_info['pixel_size'],
        target_raster_path=target_raster_path,
        resample_method='near',
        target_bb=shapely.geometry.shape(aoi_geom).bounds,
    )
    LOGGER.info('Finished warping')
    LOGGER.info(f"Clipped raster available at {target_raster_path}")
    print('ending')

    # warp the clipped raster to the target coordinate system
    #pygeoprocessing.warp_raster(
    #    base_raster_path
    #)

threads = []

def _submit_form(*args, **kwargs):
    LOGGER.info('foo')
    submit_button.description = "Clipping ..."
    submit_button.disabled = True
    #thread = threading.Thread(
    #    target=compute,
    #    args=[
    #        'intermediate/global_dem.tif',
    #        draw_control.data[0]['geometry'],
    #        4326,
    #        'foo.tif'
    #    ]
    #)
    #thread.start()
    #thread.join()
    args=[
            'intermediate/global_dem.tif',
            draw_control.data[0]['geometry'],
            4326,
            'foo.tif'
        ]
    compute(*args)
    
    submit_button.disabled = False
    submit_button.description = 'Clip layer to AOI'
    print('bar')
    
    

submit_button.on_click(_submit_form)


display(m)
display(output_handler.out)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

http://localhost:8889/notebooks/demo.ipynb


In [17]:
# Remaining TODOs for this application:
# - [ ] get the URL when running in Voila mode
# - [ ] expand this to operate on various layers available
# - [ ] Allow the user to define their target EPSG code
# - [ ] If the EPSG code represents a projected coord system (osr.IsGeographic()), provide the pixel size